In [71]:
TEST_DIR = '../test/'
IN_SHAPE_DIR = '../test/in_shape/'
GRAPH_DIR = '../test/graph'

# Preprocess route data

In [2]:
import fiona
from fiona.crs import from_epsg
from shapely.geometry import Point, LineString, shape, mapping
import rtree
from shapely.ops import snap, transform
import copy
import pyproj
from functools import partial

In [3]:
def snap_linestrings(r, lines):
    # create an empty spatial index object
    index = rtree.index.Index()
    
    snapped = []
    
    # populate the spatial index
    for index_id, geom in enumerate(lines):
        index.insert(index_id, geom.bounds)
    
    # create snapped lines
    for search_id, geom in enumerate(lines):
        
        # buffer the line
        buffered = geom.buffer(r)

        # get list of fids where bounding boxes intersect
        index_ids = [int(i) for i in index.intersection(buffered.bounds)]

        # access the features that those fids reference
        new_geom = copy.deepcopy(geom)
        for index_id in index_ids:
            
            if search_id == index_id:
                continue
            
            new_geom = snap(new_geom, lines[index_id], r)
            
            index.delete(search_id, geom.bounds)
        
        snapped.append(new_geom)
    
    return snapped

In [4]:
def write_shape(geometries, data, schema, path):
    with fiona.open(path, 'w', crs=from_epsg(4326), driver='ESRI Shapefile', schema=schema) as output:
        for i, g in enumerate(geometries):
            # attributes
            attributes = data[i]
            # write the row (geometry + attributes in GeoJSON format)
            output.write({'geometry': mapping(g), 'properties': attributes})

In [5]:
def project_to_utm(geoms):
    project_to_utm = partial(
        pyproj.transform, 
        pyproj.Proj(init='epsg:4326'),
        pyproj.Proj(init="epsg:28355", proj="utm", zone=55)
    )
    return [transform(project_to_utm, geom) for geom in geoms]

def project_from_utm(geoms):
    project_from_utm = partial(
        pyproj.transform, 
        pyproj.Proj(init="epsg:28355", proj="utm", zone=55),
        pyproj.Proj(init='epsg:4326')
    )
    return [transform(project_from_utm, geom) for geom in geoms]

In [78]:
# Load road shapefile
ROAD_PATH = TEST_DIR + 'ne_10m_roads_test.shp'
roadF = fiona.open(ROAD_PATH)
road_geoms = [LineString(shape(f['geometry'])) for f in roadF]
road_data = [f['properties'] for f in roadF]

# Project to UTM coordinate system
road_geoms = project_to_utm(road_geoms)

# Create new snapped LineString collection
SNAPPED_FILE_NAME = 'snapped.shp'
snapped = snap_linestrings(500, road_geoms)

# test_output = project_from_utm(snapped)
# write_shape(test_output, road_data, roadF.schema, TEST_DIR + SNAPPED_FILE_NAME)

# Preprocess livestock and crop data into points

In [7]:
import rasterio
import numpy as np
from affine import Affine
from collections import OrderedDict

In [9]:
# Load livestock raster
LIVESTOCK_PATH = TEST_DIR + 'csiro_bvmeat_feed_00_test.tif'

# Read raster
with rasterio.open(LIVESTOCK_PATH) as r:
    T0 = r.transform  # upper-left pixel corner affine transform
    p1 = pyproj.Proj(r.crs)
    A = r.read()  # pixel values

# Get affine transform for pixel centres
T1 = T0 * Affine.translation(0.5, 0.5)
# Function to convert pixel row/column index (from 0) to lat/lon at centre
rc2en = lambda r, c: (c, r) * T1

# Potential way to vectorise:

# All rows and columns into numpy mesh grid
# cols, rows = np.meshgrid(np.arange(A.shape[2]), np.arange(A.shape[1]))

# All eastings and northings (there is probably a faster way to do this)
# lats, lons = np.vectorize(rc2en, otypes=[np.float, np.float])(rows, cols)

In [11]:
point_geoms = []
point_data = []
it = np.nditer(A, flags=['multi_index'])

# TODO this is pretty inefficient, could come up with a vectorised implementations like that in the cell above
while not it.finished:
    value = np.asscalar(it[0])
    if value > 0:
        point_geoms.append(Point(rc2en(it.multi_index[1], it.multi_index[2])))
        point_data.append(
            {'value': np.asscalar(it[0])}
        )
    it.iternext()

point_geoms = project_to_utm(point_geoms)

# test_output = project_from_utm(point_geoms)
# schema = OrderedDict({
#     'geometry': 'Point',
#     'properties': {'value': 'float:16'}
# })
# write_shape(test_output, point_data, schema, TEST_DIR + 'point_test.shp')

# Assign livestock and crop data to route locations

In [79]:
from shapely.ops import nearest_points, split, snap
from shapely.geometry import MultiPoint, MultiPolygon

In [80]:
def make_points_on_line(geom, distance):
    if geom.geom_type == 'LineString':
        num_vert = int(round(geom.length / distance))
        if num_vert == 0:
            num_vert = 1
        return [
            geom.interpolate(float(n) / num_vert, normalized=True)
            for n in range(num_vert + 1)
        ]
    elif geom.geom_type == 'MultiLineString':
        parts = [make_points_on_line(part, distance)
                 for part in geom]
        return type(geom)([p for p in parts if not p.is_empty])
    else:
        raise ValueError('unhandled geometry %s', (geom.geom_type,))

        
def split_line_by_distance(geom, distance):
    
    # Interpolated points won't necessarily intersect line, so need to buffer
    buffed = MultiPolygon([p.buffer(0.01) for p in make_points_on_line(geom, distance)])
    
    # Buffered polygons will each yield two split points
    poly_split_lines = split(geom, buffed)

    # Stitch together long with small segments between split points
    point_split_lines = []
    for i, current_line in enumerate(poly_split_lines):
        
        if i == len(poly_split_lines) - 1: # last line
            point_split_lines.append(current_line)
        elif i % 2 == 0: # 0 or even
            last_line = current_line
        else:
            point_split_lines.append(LineString(list(last_line.coords) + list(current_line.coords)))
            last_line = current_line
    
    return point_split_lines
    

def join_points_to_lines(points, lines, step):
    
    # create an empty spatial index object
    index = rtree.index.Index()
    
    snapped = []
    
    # populate the spatial index
    for index_id, geom in enumerate(lines):
        index.insert(index_id, geom.bounds)
    
    # create lines which join points to lines
    joins = []
    for search_id, geom in enumerate(points):
        
        # buffer out the point until we hit a line
        
        r = step
        index_ids = []
        
        while len(index_ids) == 0:  
            buffered = geom.buffer(r)
            index_ids = [int(i) for i in index.intersection(buffered.bounds)]
            r += step

        closest_line = lines[index_ids[0]]  # doesn't seem to work well
        
        # nearest_points method (attaches anywhere on a line)
        # join_points = nearest_points(closest_line, geom)
        # joins.append(LineString(join_points))
        
        # snap method (only attaches to line endpoints)
        closest_point_on_line = snap(geom, closest_line, r*100)
        joins.append(LineString((geom, closest_point_on_line)))
    
    return joins

In [81]:
# Split out network every 10km

split_lines = []
for s in snapped:
    split_lines.extend(split_line_by_distance(s, 10000))

schema = OrderedDict({
    'geometry': 'LineString',
    'properties': {}
})

# test_output = project_from_utm(split_lines)
# write_shape(test_output, [{} for b in test_output], schema, TEST_DIR + 'line_split_test.shp')

In [82]:
# Join our points to the network
joins = join_points_to_lines(point_geoms, split_lines, 1000)

# back = project_from_utm(joins)
# schema = OrderedDict({
#     'geometry': 'LineString',
#     'properties': {'value': 'float:16'}
# })
# write_shape(back, point_data, schema, TEST_DIR + 'join_test.shp')

# Convert to graph and run analysis

In [83]:
import networkx as nx

In [84]:
# Write our preprocessed data to our in shape folder

back = project_from_utm(joins)
schema = OrderedDict({
    'geometry': 'LineString',
    'properties': {'value': 'float:16'}
})
write_shape(back, point_data, schema, IN_SHAPE_DIR + 'livestock_edges.shp')

back = project_from_utm(split_lines)
schema = OrderedDict({
    'geometry': 'LineString',
    'properties': {}
})
write_shape(back, [{} for b in back], schema, IN_SHAPE_DIR + 'road_edges.shp')

In [85]:
Groad = nx.read_shp(IN_SHAPE_DIR)
nx.write_shp(Groad, GRAPH_DIR)

In [87]:
Groad.nodes()

NodeView(((148.24732818407236, -41.513903951489986), (148.29704478162978, -41.58056543034832), (148.27958093467913, -41.6624509517404), (148.26959149198413, -41.74829160486112), (148.258342627582, -41.829398637703555), (148.2806710509073, -41.890945285281795), (148.20352723806442, -41.949609432983), (148.1122306339157, -41.993474946582694), (148.05348035109327, -42.05420317225051), (148.0534803398829, -42.05420326189036), (146.65117678617202, -41.526277218140386), (146.6703331781777, -41.60577012864953), (146.73251980976465, -41.68117222693026), (146.70110970548916, -41.76483406388643), (146.68185298716298, -41.848049723241076), (146.68013588992596, -41.9330749205944), (146.71619912166022, -41.99075941008691), (146.81615357450846, -42.02945667843557), (146.8161536748439, -42.02945672858957), (145.83454118724626, -41.047844240991964), (145.75827473187272, -41.11586362385832), (145.67116743714246, -41.169981208384556), (145.604587735815, -41.24332020174192), (145.57955480271903, -41.2960

In [90]:
try:
    n = nx.shortest_path_length(Groad.reverse(copy=False),(148.24732818407236, -41.513903951489986))
    print(n)
except nx.NetworkXNoPath:
    print('No path')

{(148.24732818407236, -41.513903951489986): 0, (148.2473281121611, -41.513904023418654): 1, (148.18233443251157, -41.57989560841711): 2, (148.0670827229154, -41.43782933566666): 2, (148.15041605625387, -41.52116266899999): 2, (148.23374938959233, -41.521162669000006): 2, (148.0632093931757, -41.584336059149734): 3, (147.90041605623847, -41.437829335666656): 3, (147.98374938957693, -41.521162669): 3, (148.0670827229154, -41.52116266899999): 3, (148.15041605625387, -41.60449600233333): 3, (147.96592547217344, -41.63826473897577): 4, (147.73374938956155, -41.43782933566667): 4, (147.8170827229, -41.437829335666656): 4, (147.8170827229, -41.521162669): 4, (147.90041605623847, -41.52116266899999): 4, (147.98374938957693, -41.60449600233333): 4, (148.0670827229154, -41.60449600233332): 4, (148.0670827229154, -41.687829335666656): 4, (148.15041605625387, -41.687829335666656): 4, (147.87792641090292, -41.69965262542657): 5, (147.6504160562231, -41.52116266899999): 5, (147.73374938956155, -41.5